<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-02 11:57:01
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.35 C
-------------------
Today PnL: -1.93 L (-1.42%)
Current PnL: -34.91 L (-22.26%)
CY Booked + Current PnL: -20.17 L (-12.86%)
-------------------
Total profit:  81.27 K
Total loss:  -35.72 L
-------------------
Total Booked + Current PnL: 6.81 L (5.25%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.95%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 1.00 C (74.61%)
Deployed:  1.30 C
Current:  1.35 C
CAGR/XIRR %: 1.88%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,484.83,-5.58,2.34,16.50,19.23,25406.0,3516.0,153976.0,19.88,48.0,L-LC,12.43,182.0,0.14,1.14,24.11,XY25,ATH,MINING
77,TTKPRESTIG,770.00,-2.52,-26.76,36.66,0.09,27059.0,-26965.0,73812.0,71.40,32.0,M-SC,6.52,253.0,-1.00,0.55,0.00,OX40N,NTT,DURABLES
50,MASFIN,398.61,-5.84,-7.47,31.90,22.05,28921.0,-7320.0,90660.0,-20.05,43.0,H-SC,3.40,168.0,-0.25,0.67,31.78,XR,ATH,FINANCE
33,HCLTECH,1908.19,-1.96,9.53,13.06,23.83,34607.0,23061.0,264985.0,14.46,54.0,X-LC,7.34,13.0,0.67,1.97,27.13,X40,ATH,IT
51,MEDANTA,1486.00,7.60,-7.09,33.55,24.08,39571.0,-9004.0,117946.0,-14.63,48.0,X-SC,9.42,91.0,-0.23,0.88,8.03,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,MEDANTA,1486.0,7.60,-7.09,33.55,24.08,39571.0,-9004.0,117946.0,-14.63,48.0,X-SC,9.42,91.0,-0.23,0.88,8.03,XY24,NTT,HEALTHCARE
68,STARHEALTH,761.0,5.37,-8.14,60.31,47.26,155743.0,-22886.0,258237.0,25.27,66.0,H-SC,3.30,174.0,-0.15,1.92,38.36,XY24,NTT,INSURANCE
15,CAMPUS,393.0,5.27,-11.62,50.81,33.28,75342.0,-19494.0,148281.0,-28.81,55.0,M-SC,5.75,221.0,-0.26,1.10,14.93,XY24,NTT,FOOTWEAR
70,SYMPHONY,1306.0,4.06,-29.84,42.48,-0.03,51010.0,-51060.0,120081.0,-43.61,61.0,M-SC,12.91,206.0,-1.00,0.89,10.73,OX40N,NTT,DURABLES
66,SIS,528.0,2.68,-23.01,57.49,21.26,49148.0,-25543.0,85489.0,2018.13,55.0,H-SC,4.81,163.0,-0.52,0.64,15.50,OX40N,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,3033.00,-9.20,-5.52,31.46,24.20,90727.0,-16862.0,288388.0,34.71,38.0,X-SC,2.50,99.0,-0.19,2.14,20.78,X40N,NTT,FINANCE
29,GREENPANEL,537.00,-5.84,-42.50,148.93,43.14,156780.0,-77807.0,105271.0,135.79,44.0,M-SC,17.38,231.0,-0.50,0.78,2.94,XY24,NTT,MISC
50,MASFIN,398.61,-5.84,-7.47,31.90,22.05,28921.0,-7320.0,90660.0,-20.05,43.0,H-SC,3.40,168.0,-0.25,0.67,31.78,XR,ATH,FINANCE
0,5PAISA,593.00,-5.76,-37.28,80.02,12.90,109651.0,-81466.0,137030.0,101.92,42.0,H-SC,6.27,173.0,-0.74,1.02,13.94,OX40N,NTT,FINANCE
46,JIOFIN,387.00,-5.70,-22.80,61.25,24.48,126420.0,-60965.0,206400.0,-24.08,18.0,X-LC,7.66,37.0,-0.48,1.53,19.64,XY24,BTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.0,-1.07,-0.64,46.51,45.57,120900.0,-1675.0,259944.0,-36.11,65.0,H-MC,6.88,121.0,-0.01,1.93,15.43,AR,NTT,ELECTRICAL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,-0.90,-10.78,133.04,107.91,175244.0,-15921.0,131723.0,-27.44,36.0,M-SC,10.87,220.0,-0.09,0.98,0.00,OX40N,ATH,CABLES
72,TATAELXSI,9161.00,0.96,-22.54,70.45,32.03,71941.0,-29719.0,102116.0,-16.20,48.0,H-SC,6.52,157.0,-0.41,0.76,14.38,OX40N,NTT,IT
66,SIS,528.00,2.68,-23.01,57.49,21.26,49148.0,-25543.0,85489.0,2018.13,55.0,H-SC,4.81,163.0,-0.52,0.64,15.50,OX40N,NTT,MISC
48,KANSAINER,340.00,0.25,-24.58,50.46,13.47,102627.0,-66285.0,203382.0,-69.09,42.0,H-SC,4.97,159.0,-0.65,1.51,3.41,XY24,NTT,PAINTS
77,TTKPRESTIG,770.00,-2.52,-26.76,36.66,0.09,27059.0,-26965.0,73812.0,71.40,32.0,M-SC,6.52,253.0,-1.00,0.55,0.00,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,484.83,-5.58,2.34,16.5,19.23,25406.0,3516.0,153976.0,19.88,48.0,L-LC,12.43,182.0,0.14,1.14,24.11,XY25,ATH,MINING


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-1.07,-0.64,46.51,45.57,120900.0,-1675.0,259944.0,-36.11,65.0,H-MC,6.88,121.0,-0.01,1.93,15.43,AR,NTT,ELECTRICAL
84,WIPRO,420.00,1.51,-1.37,74.66,72.27,124777.0,-2321.0,167127.0,-15.15,44.0,M-LC,2.56,101.0,-0.02,1.24,7.42,XR,NTT,IT
39,INDIAMART,4810.62,0.95,-5.64,119.07,106.72,138577.0,-6953.0,116383.0,-53.30,49.0,H-SC,9.05,138.0,-0.05,0.86,16.42,AR,ATH,MISC
26,FINCABLES,1641.55,-0.90,-10.78,133.04,107.91,175244.0,-15921.0,131723.0,-27.44,36.0,M-SC,10.87,220.0,-0.09,0.98,0.00,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-2.70,-8.61,48.41,35.63,92720.0,-18044.0,191530.0,-21.61,36.0,H-MC,5.51,119.0,-0.19,1.42,6.63,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JIOFIN,387.00,-5.70,-22.80,61.25,24.48,126420.0,-60965.0,206400.0,-24.08,18.0,X-LC,7.66,37.0,-0.48,1.53,19.64,XY24,BTT,FINANCE
44,ITC,452.00,-4.64,-24.71,47.14,10.78,85296.0,-59377.0,180941.0,-55.05,18.0,X-LC,18.27,1.0,-0.70,1.34,0.00,X40,NTT,FMCG
13,BERGEPAINT,680.00,-0.12,-17.63,47.07,21.14,88140.0,-40087.0,187252.0,-23.43,19.0,X-MC,7.30,75.0,-0.45,1.39,0.00,XY24,NTT,PAINTS
12,BATAINDIA,2096.00,-0.41,-45.01,145.88,35.20,103214.0,-57917.0,70753.0,-7.29,20.0,X-SC,19.89,93.0,-0.56,0.53,0.55,X40,NTT,FOOTWEAR
6,ASIANPAINT,3460.25,0.51,-15.12,42.50,20.95,90818.0,-38078.0,213690.0,-15.95,24.0,X-LC,10.94,36.0,-0.42,1.59,15.12,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.59,1.17,-13.10,36.42,18.55,119710.0,-49545.0,328692.0,-25.24,50.0,X-LC,0.55,3.0,-0.41,2.44,11.83,X40,ATH,IT
81,VBL,671.64,0.40,-5.28,43.24,35.67,129356.0,-16684.0,299158.0,-16.55,40.0,X-LC,1.84,5.0,-0.13,2.22,7.84,X40N,ATH,FMCG
34,HINDUNILVR,2922.00,-1.85,-9.40,25.46,13.67,59296.0,-24163.0,232900.0,-16.11,43.0,X-LC,2.24,9.0,-0.41,1.73,9.95,XY25,NTT,FMCG
5,ANGELONE,3033.00,-9.20,-5.52,31.46,24.20,90727.0,-16862.0,288388.0,34.71,38.0,X-SC,2.50,99.0,-0.19,2.14,20.78,X40N,NTT,FINANCE
36,ICICIGI,2252.93,-0.53,-3.80,24.06,19.34,46315.0,-7613.0,192496.0,-24.04,34.0,X-MC,2.75,71.0,-0.16,1.43,11.53,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-3.90,-32.21,142.14,64.15,229286.0,-76641.0,161310.0,-60.24,30.0,X-SC,7.28,84.0,-0.33,1.20,0.0,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-3.28,-24.75,106.60,55.47,195162.0,-60207.0,183079.0,-33.40,35.0,X-MC,17.14,63.0,-0.31,1.36,0.0,X40N,ATH,FINANCE
10,BAJAJHLDNG,14451.52,-2.02,-6.03,37.50,29.22,67001.0,-11455.0,178670.0,-8.67,36.0,X-LC,13.85,29.0,-0.17,1.33,0.0,X40,ATH,FINANCE
13,BERGEPAINT,680.00,-0.12,-17.63,47.07,21.14,88140.0,-40087.0,187252.0,-23.43,19.0,X-MC,7.30,75.0,-0.45,1.39,0.0,XY24,NTT,PAINTS
44,ITC,452.00,-4.64,-24.71,47.14,10.78,85296.0,-59377.0,180941.0,-55.05,18.0,X-LC,18.27,1.0,-0.70,1.34,0.0,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.23,-31.06,107.19,42.84,48038.0,-20190.0,44816.0,-54.82,47.0,X-SC,21.59,83.0,-0.42,0.33,5.34,XY24,NTT,MISC
58,RELAXO,1176.00,-4.15,-51.44,213.64,52.31,150597.0,-74669.0,70491.0,-48.31,44.0,X-SC,11.46,92.0,-0.50,0.52,4.82,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-0.41,-45.01,145.88,35.20,103214.0,-57917.0,70753.0,-7.29,20.0,X-SC,19.89,93.0,-0.56,0.53,0.55,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,7.60,-7.09,33.55,24.08,39571.0,-9004.0,117946.0,-14.63,48.0,X-SC,9.42,91.0,-0.23,0.88,8.03,XY24,NTT,HEALTHCARE
35,HONAUT,58357.33,0.87,-20.00,79.15,43.32,103132.0,-32572.0,130300.0,-31.26,45.0,X-SC,9.02,90.0,-0.32,0.97,5.80,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,452.00,-4.64,-24.71,47.14,10.78,85296.0,-59377.0,180941.0,-55.05,18.0,X-LC,18.27,1.0,-0.70,1.34,0.00,X40,NTT,FMCG
75,TMPV,600.00,-0.50,-28.25,71.40,22.98,115720.0,-63825.0,162073.0,-25.52,48.0,X-LC,4.38,2.0,-0.55,1.20,3.61,XY24,NTT,AUTO
73,TCS,4311.59,1.17,-13.10,36.42,18.55,119710.0,-49545.0,328692.0,-25.24,50.0,X-LC,0.55,3.0,-0.41,2.44,11.83,X40,ATH,IT
81,VBL,671.64,0.40,-5.28,43.24,35.67,129356.0,-16684.0,299158.0,-16.55,40.0,X-LC,1.84,5.0,-0.13,2.22,7.84,X40N,ATH,FMCG
42,INFY,1972.00,-0.46,11.19,20.73,34.24,72123.0,35026.0,347914.0,-13.50,49.0,X-LC,7.13,6.0,0.49,2.59,20.40,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,-0.84,-34.23,102.05,32.89,51819.0,-26428.0,50778.0,-727.32,49.0,L-MC,11.27,259.0,-0.51,0.38,39.84,XR,NTT,BANKS
18,COALINDIA,484.83,-5.58,2.34,16.50,19.23,25406.0,3516.0,153976.0,19.88,48.0,L-LC,12.43,182.0,0.14,1.14,24.11,XY25,ATH,MINING
68,STARHEALTH,761.00,5.37,-8.14,60.31,47.26,155743.0,-22886.0,258237.0,25.27,66.0,H-SC,3.30,174.0,-0.15,1.92,38.36,XY24,NTT,INSURANCE
33,HCLTECH,1908.19,-1.96,9.53,13.06,23.83,34607.0,23061.0,264985.0,14.46,54.0,X-LC,7.34,13.0,0.67,1.97,27.13,X40,ATH,IT
42,INFY,1972.00,-0.46,11.19,20.73,34.24,72123.0,35026.0,347914.0,-13.50,49.0,X-LC,7.13,6.0,0.49,2.59,20.40,X40,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SATIN,274.00,0.11,-14.42,77.42,51.83,183058.0,-39836.0,236448.0,-23.78,61.0,H-SC,3.11,148.0,-0.22,1.76,15.18,XY24,NTT,FINANCE
27,GILLETTE,11206.78,-1.24,2.68,28.96,32.42,75499.0,6804.0,260700.0,-15.63,64.0,X-SC,12.66,87.0,0.09,1.94,18.18,X40,ATH,FMCG
1,ABB,7934.00,-1.07,-0.64,46.51,45.57,120900.0,-1675.0,259944.0,-36.11,65.0,H-MC,6.88,121.0,-0.01,1.93,15.43,AR,NTT,ELECTRICAL
33,HCLTECH,1908.19,-1.96,9.53,13.06,23.83,34607.0,23061.0,264985.0,14.46,54.0,X-LC,7.34,13.0,0.67,1.97,27.13,X40,ATH,IT
68,STARHEALTH,761.00,5.37,-8.14,60.31,47.26,155743.0,-22886.0,258237.0,25.27,66.0,H-SC,3.30,174.0,-0.15,1.92,38.36,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.91
1,25,44.95
2,50,76.77


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.97
MC    30.60
LC    24.37
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.31
X40      24.13
X40N     14.81
AR        8.87
XR        8.73
XY25      8.67
OX40N     7.50
SR        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.95
H-SC    23.07
X-LC    20.87
M-SC    10.99
X-SC    10.17
H-MC     4.80
M-MC     1.47
M-LC     1.24
L-LC     1.14
H-LC     1.12
L-SC     0.74
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.27,-15.23,54.51
FINANCE,13.07,-18.79,65.57
IT,13.02,-21.95,82.87
MISC,6.89,-39.93,97.17
ELECTRICAL,5.98,-16.49,59.00
PAINTS,5.43,-26.33,45.10
INSURANCE,4.89,-3.05,38.31
PHARMA,4.03,-8.09,42.12
BANKS,2.88,-38.29,122.09


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3429742.0,21
AR,1374445.0,10
XR,1317940.0,12
X40,1261230.0,15
X40N,1125515.0,10
OX40N,783546.0,10
XY25,453218.0,6
SR,295496.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3690543.0,24
X-MC,1806491.0,16
M-SC,1577794.0,15
X-LC,1081344.0,12
X-SC,930017.0,9
H-MC,431843.0,3
L-SC,190542.0,2
M-LC,124777.0,1
H-LC,82412.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1260067.0      6
           AR         947251.0      5
M-SC       XY24       900013.0      6
H-SC       XR         851667.0      7
X-MC       X40        612962.0      7
           XY24       492404.0      3
X-LC       X40        469555.0      6
X-MC       X40N       455369.0      4
X-SC       X40N       434409.0      4
M-SC       OX40N      352347.0      5
H-SC       OX40N      336062.0      4
X-SC       XY24       316895.0      3
H-SC       SR         295496.0      2
X-MC       XY25       245756.0      2
X-LC       XY24       242140.0      2
           X40N       235737.0      2
H-MC       XY24       218223.0      1
           AR         213620.0      2
M-SC       XR         194272.0      2
X-SC       X40        178713.0      2
X-LC       XY25       133912.0      2
M-SC       AR         131162.0      2
M-LC       XR         124777.0      1
L-SC       XR          95405.0      1
           OX40N       95137.0      1
H-LC       AR          82412.0      1
L-MC       XR          51819.0      1
M-MC       XY25        48144.0      1
L-LC       XY25        25406.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
